In [1]:
%load_ext autoreload
%load_ext autotime

time: 284 µs (started: 2024-10-09 16:58:14 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 1.23 ms (started: 2024-10-09 16:58:14 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

time: 1.32 ms (started: 2024-10-09 16:58:14 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-unet3d-dlio-v100-node-16-v2"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/unet3d-dlio-v100-node-16-v2"
percentile = 0.99
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/unet3d/dlio-v100/node-16/v2/RAW/*.pfw.gz"


time: 1.2 ms (started: 2024-10-09 16:58:14 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 9.86 s (started: 2024-10-09 16:58:14 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    percentile=cfg.percentile,
    view_types=cfg.view_types,
)

time: 4min 25s (started: 2024-10-09 16:58:24 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          51.55 seconds                                                                                 │
│  I/O Time         2.75 seconds                                                                                  │
│                   ├── Read - 0.26 seconds (9.44%)                                                               │
│                   ├── Write - 0.00 seconds (0.02%)                                                              │
│                   └── Metadata - 2.50 seconds (90.83%)                                                          │
│  I/O Operations   10,366,437 ops                                                                                │
│                   ├── Read - 267,648 ops (2.58%)                                                                │
│                   ├── Write - 4,003 ops (0.04%)                                                                 │
│                   └── Metadata - 10,056,898 ops (97.01%)                                                        │
│  I/O Size         715.55 MiB                                                                                    │
│                   ├── Read - 715.38 MiB (99.98%)                                                                │
│                   └── Write - 175.93 kiB (0.02%)                                                                │
│  Read Requests    4 kiB-64 kiB - 267,130 ops                                                                    │
│                   ├── <4 kiB - 191,005 ops (71.50%)                                                             │
│                   ├── 4-16 kiB - 33,033 ops (12.37%)                                                            │
│                   └── 16-64 kiB - 43,092 ops (16.13%)                                                           │
│  Write Requests   4 kiB-4 kiB - 4,003 ops                                                                       │
│                   └── <4 kiB - 4,003 ops (100.00%)                                                              │
│  Nodes            16 nodes                                                                                      │
│                   ├── 16317731584986882421 - 2.75 s (100.00%) - 44.71 MiB/10.56 kiB R/W (6.25/0.00%) - 647,899  │
│                   │   ops (6.25%)                                                                               │
│                   ├── 1995665909820648179 - 2.71 s (98.67%) - 44.72 MiB/10.56 kiB R/W (6.25/0.00%) - 647,899    │
│                   │   ops (6.25%)                                                                               │
│                   ├── 3084974796030094918 - 2.71 s (98.46%) - 44.70 MiB/10.56 kiB R/W (6.25/0.00%) - 647,899    │
│                   │   ops (6.25%)                                                                               │
│                   ├── 13943161580101246997 - 2.71 s (98.38%) - 44.71 MiB/10.56 kiB R/W (6.25/0.00%) - 647,908   │
│                   │   ops (6.25%)                                                                               │
│                   ├── 12578331172924968642 - 2.70 s (98.30%) - 44.72 MiB/10.56 kiB R/W (6.25/0.00%) - 647,898   │
│                   │   ops (6.25%)                                                                               │
│                   ├── 4805284303638872715 - 2.68 s (97.59%) - 44.72 MiB/10.56 kiB R/W (6.25/0.00%) - 647,899    │
│                   │   ops (6.25%)                                                                               │
│                   ├── 17424505939907203644 - 2.68 s (97.56%) - 44.71 MiB/10.56 kiB R/W (6.25/0.00%) - 647,897   │
│                   │   ops (6.25%)                     

time: 10.4 s (started: 2024-10-09 17:02:50 -07:00)
